In [65]:
import numpy as np
import pymorton as pm
import math

In [76]:
# read txt files
coords = input('please input your coordination file:')
offs = input('please input your offsests file:')
coordinates = []
with open(coords,'r') as file:
    for line in file:
        new_line = line.split(',')
        new_line_float = list(map(float, new_line))
        coordinates.append(new_line_float)

offsets = []
with open(offs,'r') as file:
    for line in file:
        new_line = line.split(',')
        new_line_int = list(map(int, new_line))
        offsets.append(new_line_int)

# combine coords and offsets to read the polygons
# calculate MBR and global MBR of the polygons
polygonsMBR = []

def MBR(coords):
    min_x, min_y = np.min(coords, axis=0)
    max_x, max_y = np.max(coords, axis=0)
    return [min_x, max_x, min_y, max_y]

def global_MBR(coordset):
    xmin_MBR = []
    xmax_MBR = []
    ymin_MBR = []
    ymax_MBR = []

    for i in coordset:
        xmin_MBR.append(i[0])
        xmax_MBR.append(i[1])
        ymin_MBR.append(i[2])
        ymax_MBR.append(i[3])
    
    return [min(xmin_MBR), max(xmax_MBR), min(ymin_MBR), max(ymax_MBR)]


for i in offsets:
    poly_id = i[0]
    startOffset = i[1]
    endOffset = i[2]+1
    poly = coordinates[startOffset:endOffset]
    theMBR = MBR(poly)
    polygons = [poly_id, theMBR]
    polygonsMBR.append(polygons)

please input your coordination file:coords.txt
please input your offsests file:offsets.txt


In [77]:
# calculate geometric centers
# compute the z-order value of the centers (interleave_latlng)
polygonsMBRunsort = []

for i in polygonsMBR:
    ave_x = (i[1][0]+i[1][1])/2
    ave_y = (i[1][2]+i[1][3])/2
    Geocenter_zscore = pm.interleave_latlng(ave_y,ave_x)
    polygons = [i[0],i[1],Geocenter_zscore]
    polygonsMBRunsort.append(polygons)

# sort the z-order value of the MBR
polygonsMBRsorted = sorted(polygonsMBRunsort, key=lambda x:x[2]) #with zscore

MBRsorted = [] #without zscore
for i in polygonsMBRsorted:
    polygons = i[:2]
    MBRsorted.append(polygons)

In [78]:
#build Rtree

In [79]:
# set the node capacity for the Rtree
nodecapacityM = 20
nodecapacitym = 0.4*nodecapacityM

#calculate the numbers of leaf node in the Rtree
leafnodeNo = math.ceil(len(MBRsorted)/nodecapacityM)

#calculate the numbers and levels of nonleaf node part of the Rtree
nonleafnodeNo = math.ceil(leafnodeNo/nodecapacityM)
nonleafnode_level = [nonleafnodeNo]
while nonleafnodeNo > 1:
    nonleafnodeNo = math.ceil(nonleafnodeNo/nodecapacityM)
    nonleafnode_level.append(nonleafnodeNo)

Rtreelevel = len(nonleafnode_level)+1

In [80]:
#print vertices per level of the Rtree
print(str(leafnodeNo) + ' nodes at level 0')

L = 1
for i in nonleafnode_level:
    print(str(i) + ' nodes at level ' + str(L))
    L += 1

500 nodes at level 0
25 nodes at level 1
2 nodes at level 2
1 nodes at level 3


In [81]:
#construct the leaf nodes of the Rtree
#assume that each R-tree node has maximum capacity 20 and minimum number of entries 0.4*20 = 8
Rtree_leaf = []
idNo = 0

if 0 < len(MBRsorted)%nodecapacityM < nodecapacitym:
    MBRsorted_cut = MBRsorted[:int(-nodecapacitym)]
    MBRsorted_tail = [MBRsorted[int(-nodecapacitym):]]
    nodelist0 = [MBRsorted_cut[i:i+20] for i in range (0, len(MBRsorted_cut), nodecapacityM)]
    nodelist0.extend(MBRsorted_tail)
    
else:
    nodelist0 = [MBRsorted[i:i+20] for i in range (0, len(MBRsorted), nodecapacityM)]
    
    
for i in nodelist0:
    leafnode = [0, idNo, i]
    Rtree_leaf.append(leafnode)
    idNo += 1

In [82]:
# construct the upper levels of the tree
def build_rtree(lst):
    theMBR = []
    Rtree_nonleaf0 = []
    global idNo
    nodeid = 0
    for i in lst:
        nodeMBR = []
        for j in i[2]:
            nodeMBR.append(j[1])
        globalnodeMBR = [nodeid, global_MBR(nodeMBR)]
        theMBR.append(globalnodeMBR)
        nodeid += 1
        
    if 0 < len(theMBR)%nodecapacityM < nodecapacitym:
        MBRsorted_cut = theMBR[:int(-nodecapacitym)]
        MBRsorted_tail = [theMBR[int(-nodecapacitym):]]
        nodelist1 = [MBRsorted_cut[i:i+20] for i in range (0, len(MBRsorted_cut), nodecapacityM)]
        nodelist1.extend(MBRsorted_tail)
    else:
        nodelist1 = [theMBR[i:i+20] for i in range (0, len(theMBR), nodecapacityM)]
    
    for i in nodelist1:
        nonleafnode = [1, idNo, i]
        Rtree_nonleaf0.append(nonleafnode)
        idNo += 1
    return Rtree_nonleaf0


idNo = len(Rtree_leaf)
Rtree = [Rtree_leaf]

#build a Rtree with level division
for i in range(Rtreelevel-1):
    Rtree_nonleaf = build_rtree(Rtree[-1])
    Rtree.append(Rtree_nonleaf)

#build a Rtree without level division
Rtree_withoutlevel = []
for i in Rtree:
    for j in i:
        Rtree_withoutlevel.append(j)
        

In [83]:
# write output text file
str_list = [str(sub_list) for sub_list in Rtree_withoutlevel]
with open('Rtree.txt','w') as f:
    for sub_list in str_list:
        f.write(sub_list + '\n')